# Data Collection
This notebook is dedicated to the collection and insertion of data into a MySQL database hosted on Google Cloud. The collection code will ultimately be transferred to a python module file that will be saved in the same subfolder as this document.

In [1]:
import requests
import time
import config
import json
import itertools
%load_ext autoreload
%autoreload 2

## Google Maps Directions API

API documentation available [here](https://developers.google.com/maps/documentation/directions/start?hl=en_US).

Python client for maps API available [here](https://github.com/googlemaps/google-maps-services-python)

In [2]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=config.api_key)

# Geocoding my address
geocode_my_result = gmaps.geocode(config.my_address)

# Store just latitude and longitude
my_geometry = geocode_my_result[0]['geometry']['location']

# Geocoding my girlfriend's address
geocode_gf_result = gmaps.geocode(config.gf_address)

# Store just latitude and longitude
gf_geometry = geocode_gf_result[0]['geometry']['location']

In [128]:
class GDirections:
    """
    A class to keep track of and display to the user only necessary parts of the Google maps searches.
    """

    
    def __init__(self, start_geometry, end_geometry, mode):
        
        """
        Parameters
        -----------
    
        start_geometry: latitude and longitude of start location
        end_geometry: latitude and longitude of end location
        mode: One of "driving", "walking", "bicycling" or "transit"
        """
        
        self._trip_start = datetime.now()
        self._directions_json = self._retrieve_google_directions_json(start_geometry, end_geometry, mode)
        self._trip_duration = self._calculate_trip_duration()
        
        
    def _retrieve_google_directions_json(self,start_geometry, end_geometry,mode):
        
        """
        Call Google Maps API to retrieve a json with directions information
        """
        
        # Call API
        directions = gmaps.directions(
            origin = start_geometry,
            destination = end_geometry,
            mode = mode,
            departure_time = self._trip_start
        )
        return directions
    
    def _calculate_trip_duration(self):
        
        """
        Calculate and return trip duration in minutes.
        
        Returns
        -------
        
        Trip duration as a string in minutes.
        
        e.g. '50 minutes'
        """
        
        # Store arrival time and convert to datetime
        arrival_time = self._directions_json[0]['legs'][0]['arrival_time']['text']
        arrival_datetime = datetime.strptime(arrival_time, '%H:%M%p')

        # Store arrival time and convert to datetime
        departure_time = self._directions_json[0]['legs'][0]['departure_time']['text']
        departure_datetime = datetime.strptime(departure_time, '%H:%M%p')

        # Calculate difference in seconds, convert to minutes
        trip_duration = int( (arrival_datetime - departure_datetime).seconds/60 )
        
        return str(trip_duration) + ' minutes'
    
    def get_trip_duration(self):
        """
        Getter method for trip_duration
        """
        
        return self._trip_duration
    
    def get_instructions(self):
        """
        RETURNS
        -------
        A dictionary with which trains/buses to take in this trip and how long each one will take.

        PARAMETERS
        ----------

        directions: [JSON] raw directions contained within self._directions_json
        """
        directions_list = []

        directions = self._directions_json[0]['legs'][0]['steps']

        for direction in directions:

            if direction.get('transit_details'):

                travel_mode = direction['transit_details']['line']['short_name']

                direction_len = direction['duration']['text']

                directions_list.append({
                    'travel_mode' : travel_mode,
                    'length' : direction_len
                })

        return directions_list
    
    def get_trip_start(self):
        return self._trip_start.strftime("%Y/%m/%d %H:%M")

In [129]:
# Trips to gf's home
transit_to_gf = GDirections(my_geometry, gf_geometry, 'transit')

# Trips to my home
transit_to_me = GDirections(gf_geometry, my_geometry, 'transit')

In [130]:
trips = [transit_to_gf, transit_to_me]

In [131]:
trips[0].get_trip_duration()

'57 minutes'

In [132]:
trips[0].get_instructions()

[{'travel_mode': 'L', 'length': '14 mins'},
 {'travel_mode': 'E', 'length': '28 mins'}]

In [133]:
trips[0].get_trip_start()

'2020/02/13 11:23'

In [127]:
# Convert date string back to datetime
datetime.strptime(db_time, "%Y/%m/%d %H:%M")

datetime.datetime(2020, 2, 13, 11, 18)

# Next step is to set up MySQL database and automate data collection to every 5 mins or so.